In [1]:
import os
from tqdm import tqdm_notebook
label_list = []
for root, dirs, files in tqdm_notebook(os.walk("frame_images_DB")):
    for file in files:
        if file.endswith(".jpg"):
            label_list.append(root.split('/')[1])
#             print(root.split('/')[1])

In [2]:
# list number of folders contain frames
test_list = list(set(label_list))
folder_set = set()
for root, dirs, files in tqdm_notebook(os.walk("frame_images_DB")):
    for file in files:
        if file.endswith(".jpg"):
            name = root.split('/')[1]            
            if name in test_list:
                folder_set.add(root)
                test_list.remove(name)

In [3]:
len(folder_set)

1595

In [4]:
# Randomly select frames, 48 per person
import random
file_list = []
for path in folder_set:
    file = random.sample(os.listdir(path),48)
    for i in file:
        file_list.append(path+'/'+i)

In [5]:
len(file_list)

76560

In [6]:
file_list[1]

'frame_images_DB/Laurie_Laychak/0/0.750.jpg'

In [7]:
# split file path into train and test
from sklearn.model_selection import train_test_split
train_addrs,test_addrs = train_test_split(file_list, test_size = 0.2, random_state = 1004)

In [8]:
len(train_addrs)

61248

In [9]:
# generate shape
train_shape = (len(train_addrs), 24, 24, 3)
test_shape = (len(test_addrs), 24, 24, 3)

In [10]:
# generate labels
train_labels = [i.split('/')[1] for i in train_addrs]
test_labels = [i.split('/')[1] for i in test_addrs]

### HDF5

In [11]:
# Initialize hdf5
import numpy as np
import h5py
hdf5_path = 'dataset.hdf5'
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_img", train_shape, np.int8)
hdf5_file.create_dataset("test_img", test_shape, np.int8)

<HDF5 dataset "test_img": shape (15312, 24, 24, 3), type "|i1">

In [12]:
hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
dt = h5py.special_dtype(vlen=str) 
hdf5_file.create_dataset("train_labels", (len(train_addrs),), dt)
hdf5_file["train_labels"][...] = train_labels
hdf5_file.create_dataset("test_labels", (len(test_addrs),), dt)
hdf5_file["test_labels"][...] = test_labels

### Load Images

In [13]:
import cv2
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in tqdm_notebook(range(len(train_addrs))):
    # read an image and resize to (24, 24)
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (24, 24), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here

    # save the image and calculate the mean so far
    hdf5_file["train_img"][i, ...] = img[None]
    mean += img / float(len(train_labels))

# loop over test addresses
for i in tqdm_notebook(range(len(test_addrs))):
    # read an image and resize to (24, 24)
    # cv2 load images as BGR, convert it to RGB
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (24, 24), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here

    # save the image
    hdf5_file["test_img"][i, ...] = img[None]
# save the mean and close the hdf5 file
hdf5_file["train_mean"][...] = mean
hdf5_file.close()

### Read Data

In [14]:
import h5py
import numpy as np
hdf5_path = 'dataset.hdf5'
subtract_mean = False
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")
# subtract the training mean
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
# Total number of samples
data_num = hdf5_file["train_img"].shape[0]

In [15]:
from keras.preprocessing.image import array_to_img
img_test = array_to_img(hdf5_file["train_img"][0])
img_test

Using TensorFlow backend.


In [16]:
data_num

61248

In [18]:
# convert labels to OHE
from keras import utils
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(hdf5_file["train_labels"])
encoded_train = encoder.transform(hdf5_file["train_labels"])
encoded_test = encoder.transform(hdf5_file["test_labels"])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_train = utils.to_categorical(encoded_train)
dummy_test = utils.to_categorical(encoded_test)


# Build NN Structure
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

batch_size = 1280
num_classes = dummy_train.shape[1]
epochs = 3
input_shape = (24, 24, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation ='relu',
                 input_shape =input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()


history1 = model.fit(hdf5_file["train_img"], dummy_train,
                     batch_size = batch_size,
                     epochs = epochs,
                     verbose = 2,
                     shuffle = 'batch'
                     )
                     


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________